In [1]:
import math
from collections import namedtuple
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from copy import deepcopy
from tqdm import tqdm
from tqdm.notebook import tqdm
import random

import numba as nb
# from numba import jit, vectorize, float64

In [2]:
with open('./data/tsp_33810_1', 'r') as input_data_file:
    input_data = input_data_file.read()
    
lines = input_data.split('\n')
nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    # points.append(Point(float(parts[0]), float(parts[1])))
    points.append((float(parts[0]), float(parts[1])))
points = np.array(points)

In [3]:
@nb.jit(nopython=True)
def length_distance(single_point, all_points):
    return np.sqrt((all_points[:, 0]-single_point[0])**2 + (all_points[:, 1]-single_point[1])**2)


@nb.jit(nopython=True)
def calculate_travel(graph):
    n = len(graph)
    total_distance = 0.0

    for i in range(n - 1):
        total_distance += DISTANCE_MATRIX[graph[i], graph[i + 1]]

    # Add the distance to return to the starting point
    total_distance += DISTANCE_MATRIX[graph[-1], graph[0]]

    return total_distance


@nb.jit(nopython=True)
def calculate_travel_list(graph):
    n = len(graph)
    total_distance = np.empty(n)

    for i in range(n - 1):
        total_distance[i] = DISTANCE_MATRIX[graph[i], graph[i + 1]]

    # Add the distance to return to the starting point
    total_distance[-1] = DISTANCE_MATRIX[graph[-1], graph[0]]

    total_distance = np.argsort(-total_distance)
    
    return total_distance
    

@nb.jit(nopython=True)
def do_2opt(point_idx1, point_idx2, path):
    
    if point_idx2 > point_idx1:
        slice_start = point_idx1
        slice_end = point_idx2
    else:
        slice_start = point_idx2
        slice_end = point_idx1

    new_path = np.hstack((path[:slice_start], path[slice_start:slice_end][::-1], path[slice_end:]))
    
    return new_path
    

def add_penalty(path):
    util_nodes = {}
    
    for idx, node in enumerate(path):
        node_out = path[idx+1] if idx+1 < nodeCount else path[0]
        dist = DISTANCE_MATRIX[node][node_out]
        penalty = (1 + PENALTY_MATRIX[node, node_out])
        util = dist / (1 + penalty)
        util_nodes[(node, node_out)] = util

    max_util = max(util_nodes.values())
    viable_candidates = [(node, node_out) for (node, node_out), value in util_nodes.items() if value == max_util]

    for node, node_out in viable_candidates:
        PENALTY_MATRIX[node, node_out] += 1
        PENALTY_MATRIX[node_out, node] += 1
        ACTIVATE[node] = 1
        ACTIVATE[node_out] = 1


def potential_gain(p1, p2, p3, p4):

    d12 = DISTANCE_MATRIX[p1][p2]
    d34 = DISTANCE_MATRIX[p3][p4] 
    d13 = DISTANCE_MATRIX[p1][p3]
    d24 = DISTANCE_MATRIX[p2][p4]
    
    ad12 = AUGMENTED_DISTANCE_MATRIX[p1][p2]
    ad34 = AUGMENTED_DISTANCE_MATRIX[p3][p4] 
    ad13 = AUGMENTED_DISTANCE_MATRIX[p1][p3]
    ad24 = AUGMENTED_DISTANCE_MATRIX[p2][p4]

    delta = d13 + d24 - d12 - d34
    augmented_delta = ad13 + ad24 - ad12 - ad34

    return delta, augmented_delta
    

def pick_next_edge(pointidx2, path):
    
    pointidx1 = edgeidx2-1
    p1 = path[pointidx1]
    p2 = path[pointidx2]
    p2_next = path[pointidx2+1] if pointidx2+1 < nodeCount else path[0]

    # Repeatedly do 2-opt for all the neighbors
    next_probable_edge = {}
    max_distance_delta = float('inf')
    
    sorted_neighbor_points = np.argsort(DISTANCE_MATRIX[p1])[1:SEARCH_AREA]
    
    for neighbor_idx in sorted_neighbor_points:
       
        pointidx4 = np.where(path==neighbor_idx)[0][0]
        pointidx3 = pointidx4-1
        
        potential_p3 = path[pointidx3]
        potential_p4 = path[pointidx4]
    
        if potential_p4 in (p1, p2, p2_next):
            continue
    
        delta, augmented_delta = potential_gain(p1, p2, potential_p3, potential_p4)
        
        next_probable_edge[pointidx4] = [delta, augmented_delta]
        # print(pointidx4, delta)

    # filter candidates with min value
    if not next_probable_edge.values():
        return -1, -1, 0

    max_delta_idx = min(next_probable_edge, key=lambda k: next_probable_edge[k][1]) # most distance decrease
    min_distance_delta = next_probable_edge[max_delta_idx][1]

    # Pick random viable candidate
    if min_distance_delta < -1e-9:
        pointidx3 = max_delta_idx-1
        pointidx4 = max_delta_idx
        min_distance_delta = next_probable_edge[max_delta_idx][0]
    else:
        # not using None as index could be 0. if None and if 0 has same effect.
        pointidx3 = -1
        pointidx4 = -1
        min_distance_delta = 0
    
    return pointidx3, pointidx4, min_distance_delta


def init_path(node_count):
    ## Keep picking the next nearest neighbor to get the path.
    
    with tqdm(total=nodeCount) as pbar:
        
        pick_next = 0
        exploring_path = np.array([], dtype=int)
        exploring_path = np.append(exploring_path, pick_next)
        
        pbar.update(1)
   
        while exploring_path.size < nodeCount:
            
            neighbor_idx = np.argsort(DISTANCE_MATRIX[pick_next])
            
            mask = np.isin(neighbor_idx, exploring_path)
            neighbor_idx = neighbor_idx[~mask]
        
            pick_next = neighbor_idx[0]
            
            exploring_path = np.append(exploring_path, pick_next)

            pbar.update(1)

    total_distance = calculate_travel(exploring_path)

    return exploring_path, total_distance

In [4]:
ACTIVATE = np.ones(nodeCount, dtype=int)

PENALTY_MATRIX = np.zeros((nodeCount, nodeCount), dtype=int)

DISTANCE_MATRIX = np.zeros((nodeCount, nodeCount))
for i in tqdm(range(len(points))):
    DISTANCE_MATRIX[i] = length_distance(points[i], points)

AUGMENTED_DISTANCE_MATRIX = np.zeros((nodeCount, nodeCount))
for i in tqdm(range(len(points))):
    AUGMENTED_DISTANCE_MATRIX[i] = length_distance(points[i], points)

  0%|          | 0/33810 [00:00<?, ?it/s]

  0%|          | 0/33810 [00:00<?, ?it/s]

6762

In [5]:
current_path, current_distance = init_path(nodeCount)
optimal_path = deepcopy(current_path)
optimal_distance = current_distance

  0%|          | 0/33810 [00:00<?, ?it/s]

In [7]:
alpha = 0.1
lambda_factor = alpha * current_distance/nodeCount
SEARCH_AREA = nodeCount//10


lambda_factor, SEARCH_AREA, current_distance

(244.5402021337388, 3381, 82679042.34141709)

In [9]:
for _ in tqdm(range(100000)):
    
    while sum(ACTIVATE) > 0:
        for p2 in range(nodeCount):
            
            if not ACTIVATE[p2]:
                continue

            edgeidx2 = np.where(current_path==p2)[0][0]
            edgeidx1 = edgeidx2-1
            p1 = current_path[edgeidx1]
    
            # edgeidx3, edgeidx4, distance_delta = get_best_swap(p1, p2, current_path)
            edgeidx3, edgeidx4, distance_delta = pick_next_edge(edgeidx2, current_path)

    
            if edgeidx4 == -1:
                ACTIVATE[p2] = 0
                continue
    
            p3 = current_path[edgeidx3]
            p4 = current_path[edgeidx4]
            
            ACTIVATE[p1] = 1
            ACTIVATE[p2] = 1
            ACTIVATE[p3] = 1
            ACTIVATE[p4] = 1
                
            current_path = do_2opt(edgeidx2, edgeidx4, current_path)
                
            # current_distance = calculate_travel(current_path)
            current_distance += distance_delta
            
            if current_distance < optimal_distance:
                optimal_distance = current_distance
                optimal_path = deepcopy(current_path)
           
    add_penalty(current_path)
    AUGMENTED_DISTANCE_MATRIX = DISTANCE_MATRIX + (lambda_factor * PENALTY_MATRIX)
    
    # if _%100 == 0:
    print(_, optimal_distance, lambda_factor)

  0%|          | 0/100000 [00:00<?, ?it/s]

MemoryError: Unable to allocate 8.52 GiB for an array with shape (33810, 33810) and data type float64

In [ ]:
output_data = '%.2f' % optimal_distance + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, optimal_path))

with open(r'./results/tsp_574_1.txt', 'w') as file:
    # Write the Python code to the file
    file.write(output_data)

In [ ]:
ACTIVATE

In [ ]:
optimal_distance

In [ ]:
p2 = 5
edgeidx2 = np.where(optimal_path==p2)[0][0]
edgeidx1 = edgeidx2-1
p1 = current_path[edgeidx1]

edgeidx3, edgeidx4, distance_delta = get_best_swap(p1, p2, optimal_path)

In [ ]:
new_path = do_2opt(edgeidx2, edgeidx4, optimal_path)
new_distance = calculate_travel(new_path)
new_distance

In [ ]:
optimal_distance

In [ ]:
edge_x = []
edge_y = []

for idx in tqdm(list(optimal_path) + [list(optimal_path)[0]]):
    
    x0, y0 = points[idx]
    
    edge_x.append(x0)
    edge_y.append(y0)

In [ ]:
fig = go.Figure(data=go.Scattergl(
    x=edge_x,
    y=edge_y,
    mode="markers+lines",
    # mode="markers"
))
fig.update_layout(height=750)
fig.show()
